In [92]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import LabelEncoder

from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
from sklearn.metrics import confusion_matrix
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [90]:
data = pd.read_csv('../data/Breast_Cancer.csv')
print(f'Original dataset: {data.shape}')

def check_binary(df):
    return [column for column in df.columns if df[column].isin(['Yes','No']).any()]

def check_pseudobi(df):
    return [column for column in df.columns if df[column].isin(np.arange(0.0, 5.0)).any()]

def check_marital(df):
    return [column for column in df.columns if df[column].isin(np.unique(data['marital'])).any()]

def getcolumns_rate(dataframe, rate = 0.25): 
  less_than_50pct_nonnull = (dataframe.isnull().sum() / dataframe.shape[0]) < rate
  return dataframe.columns[less_than_50pct_nonnull]

def get_objectColumns(dataframe, type = 'object'):
    return list(dataframe.select_dtypes(include = type).columns)

def variance_inflation(df):
    vif = pd.DataFrame()
    vif['VIF'] = [variance_inflation_factor(data.values, i) for i in range(data.shape[1])]
    vif['Features'] = data.columns
    vif.sort_values(by = 'VIF', ascending = False)

remove_columns = getcolumns_rate(data)
data = data[remove_columns]
data = data.dropna()
encoder = LabelEncoder()

data['bdate'] = data['bdate'].str[-4:]
data['age']   = 2566 - data['bdate'].astype('int')
data = data.drop('bdate', axis = 1)

encode_columns = get_objectColumns(data)
for column in encode_columns:
    data[column] = encoder.fit_transform(data[column])

data['noova'] = data['noova'].replace([9.0], [3.0], inplace = False)
encode_columns = ['nobreast', 'nosecon', 'noova', 'inj', 'noparity']
for column in encode_columns:
    data[column] = encoder.fit_transform(data[column])

X = data.drop('diag_cancer', axis = 1).to_numpy()
y = data['diag_cancer'].to_numpy()

Original dataset: (15718, 46)


In [94]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

print('==============Gradient Boosting Classifier=================')
clf = GradientBoostingClassifier(n_estimators = 100, learning_rate = 1.0, max_depth = 1, random_state=0)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))
y_pred = clf.predict(X_test)
print(confusion_matrix(y_test, y_pred))

print('=============XGBoost Classifier===============')
xgb_model = xgb.XGBClassifier(n_estimators=100,eta=0.3, gamma=0.5, random_state=0)
xgb_model.fit(X_train, y_train)
print(xgb_model.score(X_test, y_test))
y_pred_xgb = xgb_model.predict(X_test)
print(confusion_matrix(y_test, y_pred_xgb))

==============Gradient Boosting Classifier=================
0.9947072444591465
[[  18   10]
 [   6 2989]]
=============XGBoost Classifier===============
0.9943764472378432
[[  15   13]
 [   4 2991]]
